In [2]:
from transformers import ElectraTokenizer, ElectraModel
import torch

In [3]:
import pandas as pd
import numpy as np

In [4]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
model = ElectraModel.from_pretrained('google/electra-small-discriminator')

In [5]:
inputs = tokenizer("Hello, my dog is cute and handsome. Why is that", return_tensors="pt")
outputs = model(**inputs)

In [6]:
outputs[0][0]

tensor([[ 0.1752,  0.5045, -0.6138,  ...,  1.3884, -1.0874, -0.2617],
        [ 0.0662, -0.2479,  0.0060,  ...,  0.2193, -0.8583, -0.2739],
        [-0.2690,  0.6088, -0.7223,  ...,  0.3475, -0.1311,  0.8110],
        ...,
        [-0.7021, -0.0967,  0.4044,  ..., -0.3064, -1.2506,  0.7733],
        [-0.3638,  0.2675,  0.2307,  ..., -0.3741,  0.8406,  0.5936],
        [ 0.1733,  0.5076, -0.6140,  ...,  1.3880, -1.0873, -0.2581]],
       grad_fn=<SelectBackward>)

In [7]:
dataset =  pd.read_csv("data.csv")

In [8]:
def get_tokenization(row):
    return tokenizer(str(row["title"])+str(row["content"]), return_tensors="pt")

In [9]:
def get_average_vector(tokens):
    return torch.mean(model(**tokens)[0][0],axis=1)

In [10]:
dataset["tokenized"]=dataset.apply(lambda x: get_tokenization(x), axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4259 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (718 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (882 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (862 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [11]:
dataset["vector_representation"]=dataset.apply(lambda x: get_average_vector(x["tokenized"]), axis=1)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1150976 bytes. Buy new RAM!


In [5]:
last_hidden_states = outputs.last_hidden_state

AttributeError: 'tuple' object has no attribute 'last_hidden_state'